# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = '' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [11]:
# Write your answer here
import pandas as pd
import numpy as np
import glob
import re

DATA_FOLDER = 'Data/ebola/'

This next cell is use to suppress Pandas FutureWarnings which always come up on ambiguity from Column/Index names

In [88]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Guinea

### Check structures of the files

#### Headers

In [89]:
allFiles = glob.glob(DATA_FOLDER + 'guinea_data/*.csv')

def check_headers(files):
    list_ = []
    for file_ in files:
        df = pd.read_csv(file_, header=None)
        list_.append(df.head(n=1))

    frame = pd.concat(list_)
    # Transpose to not drop record and better visualisation
    return frame.drop_duplicates().T

check_headers(allFiles)

,0,0,0,0
0,Date,Date,Date,Date
1,Description,Description,Description,Description
2,Totals,Totals,Totals,Totals
3,Conakry,Conakry,Conakry,Conakry
4,Gueckedou,Gueckedou,Gueckedou,Gueckedou
5,Macenta,Macenta,Macenta,Macenta
6,Dabola,Dabola,Dabola,Dabola
7,Kissidougou,Kissidougou,Kissidougou,Kissidougou
8,Dinguiraye,Dinguiraye,Dinguiraye,Dinguiraye
9,Telimele,Telimele,Telimele,Telimele


#### Rows title

In [90]:
# Footers, check if the description are the same for each file
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, usecols=['Description'])
    #print(file_, df.shape)
    list_.append(df)
    
frame = pd.concat(list_, axis=1)
cleaned = frame.T.drop_duplicates().T

first_column = set(cleaned[[0]].squeeze().tolist())
second_column = set(cleaned[[1]].squeeze().tolist())
print("sizes: {}".format(list(map(str, map(len, [first_column, second_column])))))
# Extra columns
second_column - first_column

# Let's discover which files doesn't follow the pattern
# Check if data is already the same in each file

KeyError: '[0] not in index'

**Headers:** Therefore, sometimes, region are missing, but the first three columns are always available: Date, Description and Totals. If we see some disrepancies between totals and the sum of all remaining columns, we are going to collect them.

**Rows:** "New deaths registered today" and "New deaths registered" will be considerer as the same data

#### Check date per file

In [91]:
def are_date_unique_per_file():
    for file_ in allFiles:
        df = pd.read_csv(file_, usecols=['Date'])
        #print(file_, df.shape)
        if df.drop_duplicates().shape[0] > 1:
            return False
    return True

are_date_unique_per_file()

True

### Get the data

Per file, we are going to compare the Totals colun and all region (if they sum up correctly) for the row: "New deaths registered today (confirmed)" or "New deaths registered" per file


In [92]:
# Write your answer here
import pandas as pd
import glob

DATA_FOLDER = 'Data/ebola/'

#Guinea
allFiles = glob.glob(DATA_FOLDER + 'guinea_data/*.csv')
frame = pd.DataFrame()

list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=['Description', 'Date'], usecols=['Date', 'Description', 'Totals'])
    if "New deaths registered" in df.index:
        print(df.loc["New deaths registered among health workers"])

        df = df.loc["New deaths registered"]
    else:
        df = df.loc["New deaths registered today"]
    
    list_.append(df)
    
# New deaths registered among health workers always zero!
    
    

#df[["New deaths registered today (confirmed)"]]
#df.loc[df.Description == "New deaths registered today"]
frame = pd.concat(list_)
frame.columns = ["Deaths (via total)"]
frame 

#with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#frame.sort_index()
#frame.index[0]
#rame.index = frame.index.map(lambda x: ("-".join(x.split('-')[0:2])))
#rame.astype(int).groupby(['Date']).sum()/30


           Totals
Date             
2014-08-26      0
           Totals
Date             
2014-08-27      0
           Totals
Date             
2014-08-30      0
           Totals
Date             
2014-08-31      0
           Totals
Date             
2014-09-02      0
           Totals
Date             
2014-09-04      0
           Totals
Date             
2014-09-07      0
           Totals
Date             
2014-09-08      0
           Totals
Date             
2014-09-09      0
           Totals
Date             
2014-09-11      0
           Totals
Date             
2014-09-14      0
           Totals
Date             
2014-09-16      0
           Totals
Date             
2014-09-17      0
           Totals
Date             
2014-09-19      0
           Totals
Date             
2014-09-21      0
           Totals
Date             
2014-09-22      0
           Totals
Date             
2014-09-23      0
           Totals
Date             
2014-09-24      0
           Totals
Date      

,Deaths (via total)
Date,
2014-08-04,2
2014-08-26,5
2014-08-27,2
2014-08-30,5
2014-08-31,3
2014-09-02,5
2014-09-04,5
2014-09-07,4
2014-09-08,4


In [93]:
list_ = []
allFiles = glob.glob(DATA_FOLDER + 'guinea_data/*.csv')
#allFiles = ["Data/ebola/guinea_data/2014-09-22.csv"]

   
# column titles    
DEATHS = "Death"
DEATHS_MANUAL_SUM = "Death (manual sum)"
NEW_CASES = "New cases"
NEW_CASES_MANUAL_SUM = "New cases (manual sum)"
divisible_fields = [DEATHS, DEATHS_MANUAL_SUM, NEW_CASES, NEW_CASES_MANUAL_SUM]

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=['Description', 'Date']).fillna(value=0)
    
    #print(df.dtypes)
    
    
    if "New deaths registered" in df.index:
        deaths = df.loc["New deaths registered"].copy()
    else:
        deaths = df.loc["New deaths registered today"].copy()
        
    new_cases = df.loc["New cases of confirmed"].copy()
    
    #print(new_cases)

    deaths_per_region = pd.to_numeric(deaths.drop("Totals", axis=1).values[0]).sum().astype(int)
    new_cases_per_region = pd.to_numeric(new_cases.drop("Totals", axis=1).values[0]).sum().astype(int)
 
    
    deaths[DEATHS_MANUAL_SUM] = deaths_per_region
    deaths[NEW_CASES] = int(new_cases["Totals"][0])
    deaths[NEW_CASES_MANUAL_SUM] = new_cases_per_region


    
    #deaths = deaths[["Totals", "Death"]]
    #deaths.columns = ["Deaths", "Deaths (manual sum)"]
    
    deaths = deaths[["Totals", DEATHS_MANUAL_SUM, NEW_CASES, NEW_CASES_MANUAL_SUM]]
    deaths.columns= [DEATHS, DEATHS_MANUAL_SUM, NEW_CASES, NEW_CASES_MANUAL_SUM]
    
    deaths[DEATHS] = deaths[DEATHS].astype(int)
    
    #print(deaths)
    
    list_.append(deaths)
    
    
frame = pd.concat(list_)
frame["Country"] = "Guinea"
frame["Date"] = pd.to_datetime(frame.index, format="%Y-%m-%d")
frame
frame.reset_index(drop=True)
frame.set_index(["Country", "Date"])

frame["Month+Year"] = frame["Date"].apply(lambda x: x.replace(day=1))
frame["daysInMonth"] = frame["Month+Year"].apply(lambda x: int(pd.tslib.monthrange(x.year, x.month)[1]))


#frame.groupby("Month+Year").sum() / frame["daysInMonth"]
frame[divisible_fields] = frame[divisible_fields].div(frame["daysInMonth"].values,axis=0)
frame = frame.drop(["Month+Year", "daysInMonth", "Date"], axis=1)
#frame[DEATHS].values
frame

AttributeError: module 'pandas.tslib' has no attribute 'monthrange'

## Liberia

### Check structures of the files

#### Headers

In [94]:
allFiles = glob.glob(DATA_FOLDER + 'liberia_data/*.csv')

check_headers(allFiles)

,0,0,0,0,0,0,0,0,0,0
0,Date,Date,Date,Date,Date,Date,Date,Date,Date,Date
1,Variable,Variable,Variable,Variable,Variable,Variable,Variable,Variable,Variable,Variable
2,National,National,National,National,National,National,National,National,National,National
3,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County,Bomi County
4,Bong County,Bong County,Bong County,Bong County,Bong County,Bong County,Bong County,Bong County,Bong County,Bong County
5,Grand Kru,Gbarpolu County,Gbarpolu County,Gbarpolu County,Grand Bassa,Grand Bassa,Grand Bassa,Gbarpolu County,Gbarpolu County,Gbarpolu County
6,Lofa County,Grand Bassa,Grand Bassa,Grand Bassa,Grand Cape Mount,Grand Cape Mount,Grand Cape Mount,Grand Bassa,Grand Bassa,Grand Bassa
7,Margibi County,Grand Cape Mount,Grand Cape Mount,Grand Cape Mount,Grand Gedeh,Grand Gedeh,Grand Gedeh,Grand Cape Mount,Grand Cape Mount,Grand Cape Mount
8,Maryland County,Grand Gedeh,Grand Gedeh,Grand Gedeh,Lofa County,Lofa County,Lofa County,Grand Gedeh,Grand Gedeh,Grand Gedeh
9,Montserrado County,Lofa County,Lofa County,Grand Kru,Margibi County,Margibi County,Margibi County,Lofa County,Lofa County,Grand Kru


We wanted to use first the "Newly reported deaths" column, but they were disrepedancies with the cumulated data, and the newly reported data did not indicate if it was confirmed, suspected, etc. For the "Total death/s in confirmed cases",  less than half of the files didn't contain the value for this field. So either we switch to no data or more data but overestimation. Then we decided to switch to cumulated "Total death/s in confirmed, probable, suspected cases".

In [95]:
NEW_CASES = "New case/s (confirmed)"
TOTAL_NEW_CASES = "Total confirmed cases"
TOTAL_DEATH = "Total death/s in confirmed, probable, suspected cases"
NEWLY_REPORTED_DEATH = "Newly reported deaths"


# most of error are extra \n and spaces
#columns = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
def check_column_in_files(files, column_name, lower_case=False):
    #Added lower_case boolean not to rewrite function as Sierre Leone data are lowercased.
    cols=['Variable', 'Date']
    if lower_case:
        cols=['variable', 'date']
    
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=cols).fillna(value=0)
        
        #df["edited_variable"] = df.index.levels[0].values
        #df["edited_variable"] = 
        new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
        df.index = df.index.set_levels(levels=new_values, level=0)
        
        #print(df.index.levels[0])
        
        #result = False
        result = column_name in df.index
            
        
        
        if(column_name == TOTAL_DEATH):
            pass
            #print("{} {}".format(result, file_))
        if not result:
            if(column_name == TOTAL_DEATH):
                print(df.index.levels[0])
            return False
    
        
    return True


to_check = [NEW_CASES, TOTAL_DEATH, NEWLY_REPORTED_DEATH, TOTAL_NEW_CASES]
list(map(lambda x: check_column_in_files(allFiles, x), to_check))

[True, True, True, True]

So, the both lines are available in each file, however a quick check the files indicates that the cell may be empty.

Below, we found out that there is many NA values for the lasts months, only totals are available which mixes confirmed, probable and suspected cases. We have chosen along the way to consider only confirmed cases, so we are going to drop those data. First, let's check how many data we are going to drop.

Also, there is discrependencies between New deaths reported and cumulated data.

For the new cases, we also are going to use the cumulated data, except for the last file, were it seems that the cumulated data has moved to the new cases column. We patch manually this

In the `2014-10-04-v142.csv` file, there is two Total deaths line, we keep only the second one

In [96]:
allFiles = glob.glob(DATA_FOLDER + 'liberia_data/*.csv')

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=['Variable', 'Date'])
    
    # Normalize variable column
    new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
    df.index = df.index.set_levels(levels=new_values, level=0)
    
    data = df[["National"]]

    
    total_deaths = data.loc[TOTAL_DEATH]
    new_cases = data.loc[NEW_CASES]
    total_cases = data.loc[TOTAL_NEW_CASES]
    
    concatenated = pd.concat([total_deaths.tail(n=1), new_cases, total_cases], axis=1)
    concatenated.columns = ["Total death", "New cases", "New cases (cumul.)"]
    
    
    
    list_.append(concatenated)
    
    
    
frame = pd.concat(list_)
#frame.index = pd.to_datetime(frame.index, format="%m/%d/%Y")

#frame.index = frame.index.set_levels([frame.index.levels[0], pd.to_datetime(frame.index.levels[1])])
#frame = frame.swaplevel()

# keep only NaN value.
death_null = frame[frame["Total death"].isnull()].shape[0]
death_total = frame.shape[0]

cases_null = frame[frame["New cases"].isnull()].shape[0]
cases_total = frame.shape[0]
print("NaN Death ratio {} % ({}/{})".format(death_null / death_total * 100, death_null, death_total))
print("NaN New Case ratio {} % ({}/{})".format(cases_null / cases_total * 100, cases_null, cases_total))

frame["New cases (cumul.)"]["12/9/2014"] = frame["New cases"]["12/9/2014"]
#frame["New cases (cumul.)"] = 

frame.index = pd.to_datetime(frame.index)

def remove_outliers(x,y):
    #print(x, y, x > y)
    if(x > y):
        return x
    return y

frame["corr"] = frame["New cases (cumul.)"].rolling(window=2).apply(lambda x: remove_outliers(x[0], x[1]))

frame["Month"] = frame.index.map(lambda x: x.month).astype(int)
frame["Day"] = frame.index.map(lambda x: x.day).astype(int)
frame.index = [frame["Month"], frame["Day"]]

#with pd.option_context('display.max_rows', None, 'display.max_columns', 5):
    #print(frame)
    
a = pd.concat(
    [frame.groupby(['Month'])['Day'].min().reset_index(),
    frame.groupby(['Month'])['Day'].max().reset_index()]
).sort_index()
result = pd.merge(a, frame, on=["Month", "Day"], how="left")

result.index = pd.to_datetime(result[['Month','Day']].apply(lambda x : '{}-{}-2011'.format(x[0],x[1]), axis=1))
result = result.drop(["Month", "Day"], axis=1)

result

NaN Death ratio 3.0 % (3/100)
NaN New Case ratio 15.0 % (15/100)


,Total death,New cases,New cases (cumul.),corr
2011-06-16,16.0,1.0,12.0,NaN
2011-06-29,49.0,2.0,43.0,43.0
2011-07-01,61.0,2.0,48.0,48.0
2011-07-26,145.0,6.0,92.0,92.0
2011-08-02,257.0,0.0,132.0,132.0
2011-08-28,885.0,22.0,369.0,369.0
2011-09-01,1004.0,14.0,406.0,406.0
2011-09-30,2058.0,1.0,925.0,925.0
2011-10-01,2082.0,0.0,926.0,926.0
2011-10-31,2697.0,NaN,73.0,2437.0


In [97]:
def only_year_month(x):
    return str(x.year) + '-' + str(x.month)

def avg_table_daily(table, ):
    new_indexes = reversed(list(set(table.index.map(only_year_month))))
    avg_result = pd.DataFrame(index=new_indexes, columns=['Total Death (daily avg.)', 'New cases (daily avg.)'])
    index_list = table.index
    i = 0
    while i < len(index_list):
        new_ind = str(index_list[i].year) + '-' + str(index_list[i].month)
        num_days = (int(index_list[i+1].day) - int(index_list[i].day)) + 1
        avg_result['Total Death (daily avg.)'][new_ind] = (int(table['Total death'][index_list[i+1]]) - int(table['Total death'][index_list[i]])) / num_days
        avg_result['New cases (daily avg.)'][new_ind] = (int(table['New cases (cumul.)'][index_list[i+1]]) - int(table['New cases (cumul.)'][index_list[i]])) / num_days
        i =  i + 2

    return avg_result

In [98]:
avg_result_liberia = avg_table_daily(result)
avg_result_liberia

,Total Death (daily avg.),New cases (daily avg.)
2011-8,23.2593,8.77778
2011-6,2.35714,2.21429
2011-10,19.8387,-27.5161
2011-12,14.3333,13.6667
2011-7,3.23077,1.69231
2011-11,15.7586,12.6552
2011-9,35.1333,17.3


## Sierre Leone

### Check structures of the files

#### Headers

In [99]:
allFiles = glob.glob(DATA_FOLDER + 'sl_data/*.csv')
check_headers(allFiles)

,0,0,0,0,0,0,0,0
0,date,date,date,date,date,date,date,date
1,variable,variable,variable,variable,variable,variable,variable,variable
2,Kailahun,Kailahun,Kailahun,Kailahun,Kailahun,Kailahun,Kailahun,Kailahun
3,Kenema,Kenema,Kenema,Kenema,Kenema,Kenema,Kenema,Kenema
4,Kono,Kono,Kono,Kono,Kono,Kono,Kono,Kono
5,Kambia,Kambia,Kambia,Kambia,Kambia,Kambia,Kambia,Kambia
6,Koinadugu,Koinadugu,Koinadugu,Koinadugu,Koinadugu,Koinadugu,Koinadugu,Koinadugu
7,Bombali,Bombali,Bombali,Bombali,Bombali,Bombali,Bombali,Bombali
8,Tonkolili,Tonkolili,Tonkolili,Tonkolili,Tonkolili,Tonkolili,Tonkolili,Tonkolili
9,Port Loko,Port Loko,Port Loko,Port Loko,Port Loko,Port Loko,Port Loko,Port Loko


For the Sierra Leone we decided to keep the *confirmed* number for the new cases and death, as it seemed to make more sense to keep only the amount of infected and dead we were sure about.

In [100]:
TOTAL_NEW_CASES = "cum_confirmed"
TOTAL_DEATH = "death_confirmed"

to_check = [TOTAL_DEATH, TOTAL_NEW_CASES]
list(map(lambda x: check_column_in_files(allFiles, x, True), to_check))

[True, True]

As the total deaths data was missing from the last file for december (13-12-2014) we decided to keep the amounts (for every column) shown in the last file that was complete (05-12-2014). This is why we do not take the last two files in the calculations.

In [101]:
list_ = []

for file_ in allFiles[:len(allFiles) - 2]:
    df = pd.read_csv(file_, index_col=['variable', 'date'])
    
    # Normalize variable column
    new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
    df.index = df.index.set_levels(levels=new_values, level=0)
    
    data = df[["National"]]
    
    total_deaths = data.loc[TOTAL_DEATH]
    total_cases = data.loc[TOTAL_NEW_CASES]
    
    concatenated = pd.concat([total_deaths.tail(n=1), total_cases], axis=1)
    concatenated.columns = ["Total death", "New cases (cumul.)"]
    
    list_.append(concatenated)
    
    
    
frame = pd.concat(list_)
frame.index = pd.to_datetime(frame.index)
frame["Month"] = frame.index.map(lambda x: x.month).astype(int)
frame["Day"] = frame.index.map(lambda x: x.day).astype(int)
frame.index = [frame["Month"], frame["Day"]]

a = pd.concat(
    [frame.groupby(['Month'])['Day'].min().reset_index(),
    frame.groupby(['Month'])['Day'].max().reset_index()]
).sort_index()
result = pd.merge(a, frame, on=["Month", "Day"], how="left")

result.index = pd.to_datetime(result[['Month','Day']].apply(lambda x : '{}-{}-2014'.format(x[0],x[1]), axis=1))
result = result.drop(["Month", "Day"], axis=1)

result

,Total death,New cases (cumul.)
2014-08-12,264,717
2014-08-31,387,1077
2014-09-01,396,1115
2014-09-30,550,2184
2014-10-01,532,2212
2014-10-31,1070,3936
2014-11-01,1077,3996
2014-11-29,1496,5831
2014-12-01,1549,5978
2014-12-05,1669,6292


Now putting all that togheter and averaging daily we get this results for the Sierra Leone data:

In [102]:
avg_result_SL = avg_table_daily(result)
avg_result_SL

,Total Death (daily avg.),New cases (daily avg.)
2014-8,6.15,18
2014-9,5.13333,35.6333
2014-10,17.3548,55.6129
2014-11,14.4483,63.2759
2014-12,24,62.8


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here